In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#이 부분 자기 폴더 경로에 맞게 바꾸기
%cd drive/My\ Drive/Colab\ Notebooks/BERT/

/content/drive/My Drive/Colab Notebooks/BERT


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BERT/SA/sentiment_result/6.7_lda_sigmoid/' + 'topic_' + str(0) + '_sig_text.csv', lineterminator='\n')
#np.argsort(df.iloc[:, -3:].iloc[0, :])[-1], df.iloc[:, -3:].iloc[0, :]

In [ ]:
일별로 긍정 부정 중립의 감성 스코어 중 가장 값이 큰 것을 가지고 집계
긍정이 가장 크면 +스코어
+
중립이면 0
+
부정이면 -스코어

-> 이 결과를 일별 트윗 개수로 나눈다.

In [ ]:
window size 3 -> 데이터 shift 시켜서 이틀씩 겹친다. 1~3 2~4 3~5.  4면 1~4, 2~5 3~6...
상관관계 윈도우 사이즈 바꿔서 해보기

1 --- 1 2 3 4 5
2 --- 12 23 34 45
3 --- 123 234 345

논문이 주식 변동폭으로 비교했는지 봐야 함


주식 변동 폭 - 윈도우 사이즈 3이면 5.15값과 5.12값을 (5.15-5.12)/5.12로 계산하여 사용.
이거를 그냥 감성점수 집계한 것과 상관관계를 봐도 되고, 감성점수를%로 바꿔서 봐도 됨

5.15 5.16 5.17 이 아니라 5.15 5.14 5.13 으로 해야 함!!!!!!!!!!!!!!! 그러니까 마지막 데이터 수집 일짜부터 하나씩 뒤로 미루면서 봐야 한다.
만약 처음 수집 날짜까지 가버리면 5.10이라 할 때 5.9 5.8 5.7 데이터가 없으므로 이것들을 0으로 해서 패딩을 넣고 비교할 수도 있음.

기업 별로 상관관계 보면 더 좋음. -> 이거는 s&p가 종합적으로 기업을 판단한거라 하려면 기업별 주식 값과 비교해야 한다.



In [ ]:
상관관계를 찾는 8조만의 알고리즘이
될 수 있는데,
머리아프면 기존대로 중립은 고려하지 말고 해요

제가 의도한거는
가령 중립이 .5이면 긍부정 합이 .5니깐
.5만큼의 범위 내에서만 움직이게 만들먼 좋겠다는거에요

예를 들어,
토픽1, 긍.3, 부.2, 중.5
토픽2, 긍.5, 부.4, 중.1
토픽3, 긍.55, 부.45, 중0 이면,
세개의 토픽 모두 긍부정 차이는 .1이지만
그 영향은 다르다고 생각해서
(1-중립)으로 보정하면,
토픽1, .05
토픽2, .09
토픽3, .10 이렇게 긍정에서 차이가 나게끔 만들수 있고
스케일이 [-1, 1] 이니깐 그 차이는 클거라고 생각한거에요

제가 생각할때 최소한 비교해야 하는 값들은
1. 로우 값(부정 제외한 값)
2. 보정 값(보정한 값)
3. 윈도우(며칠치를 묶어서 볼건지)
4. 시프트(예측과 실제가 며칠 차이나는지)
그리고 위 방법응 섞어서(1/2 + 3, 1/2 + 4) 등

그리고 값을 비교하려면 고려해야 하는게
t-2: +3%
t-1: +1%
t: -1% 라고 하면
지금은 [3, 1, -1] 을 비교한거 같은데
비율이기 때문에[1.03, 1.043, 1.03257)을 비교하는게 맞을거 같아요(그러면 당연히 위에서 구한 값도 등락율%로 변환해야 할거구요)

그렇게 두가지 값을 위의 4가지 조합이랑 비교해보는게 필요할거 같아요
그러면 0을 기준으로 등락을 비교하는게 되니깐요


In [ ]:
날짜 별로

긍정이 가장 높은 개수 / 총 트윗 개수 --- a
부정이 가장 높은 개수 / 총 트윗 개수 --- b
중립이 가장 높은 개수 / 총 트윗 개수 --- c
a+b+c == 1

집계해서 사용: a - b + c(는 바이어스) * 0.01  -> 간접적으로 영향만 보려 하는데 어떻게 할까

원래는 각 트윗별로 가장 높은 점수를 가지고 긍정+ 부정- 중립0 으로 계산했으나
이제 a - b + c(는 바이어스)로 계산.


토픽1
5/21 - a - b + c(는 바이어스). |
5/22 - a - b + c(는 바이어스). | ----집계
5/23 - a - b + c(는 바이어스). |

토픽2
5/21 - a - b + c(는 바이어스). |
5/22 - a - b + c(는 바이어스). | ----집계
5/23 - a - b + c(는 바이어스). |

...

In [11]:
import numpy as np
import pandas as pd

# 오늘 수집한 s&p 지수: 5.23~6.15
snp = [4145.58, 4115.24, 4151.28, 4205.45, 4205.52, 4205.52, 4205.52, 4205.52, 4179.83, 4221.02, 4282.37, 4279.37,
       4276.37, \
       4273.79, 4283.85, 4267.52, 4293.93, 4298.86, 4298.86, 4298.86, 4338.93, 4369.01, 4372.59, 4425.84]

nasdaq_100 = [13672.54, 13604.48, 13938.53, 14298.41, 14298.41, 14298.41, 14298.41, 14354.99, 14254.09, 14441.51, 14546.64,\
              14546.64, 14546.64, 14556.50, 14558.09, 14303.29, 14484.54, 14528.36, 14528.36, 14528.36, 14784.30, 14900.85, 15005.69, 15185.48]
dow = [33055.51, 32799.92, 32764.65, 33093.34, 33093.34, 33093.34, 33093.34, 33042.78, 32908.27, 33061.57, 33762.76, 33762.76,\
       33762.76, 33562.86, 33573.28, 33665.02, 33833.61, 33876.78, 33876.78, 33876.78, 34066.33, 34212.12, 33979.33, 34408.06]

#날짜별로 groupby ->
#결과는 집계값/총 트윗수

#1. 전체 그래프 그려보기
#2. 시프트 값도 반드시 구현!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#-> 기준이 5.23이라면 이때의 감성점수로 5.24, 5.25...의 S&P500지수 값 예측?
#3. 기업별로도 비교

#감성분석 결과 df 넣기
def get_sentiment_score(sentiment_df):
  import pandas as pd
  import numpy as np

  sentiment_df['tweetDate'] = sentiment_df['tweetDate'].apply(lambda x: x.split()[0])
  date_group = sentiment_df.groupby('tweetDate')
  new_df = pd.DataFrame(columns = ['date', 'agg_score'])

  for group in date_group:
    total_pos = 0
    total_neg = 0
    total_neu = 0

    pos_idx = 2
    neg_idx = 0
    neu_idx = 1

    date = group[0]
    df = group[1]
    total_tweet = df.shape[0]

    for i in df.iloc[:, -3:].values:
      idx = np.argsort(i)[-1]
      if idx == pos_idx:
          total_pos += 1
      if idx == neg_idx:
          total_neg += 1
      if idx == neu_idx:
          total_neu += 1

    result = ((total_pos - total_neg) / total_tweet) * (1 - (total_neu / total_tweet))
    new_df.loc[len(new_df) + 1] = [date, result] #df에 추가
  return pd.Series(data = new_df['agg_score']).rename(new_df['date'])


def get_correlation(df, index, window):
    import numpy as np
    from scipy import stats
    import matplotlib.pyplot as plt
    sentiment_score = get_sentiment_score(df)
    window = window + 1

    agg_date = [sentiment_score.index[i - 1] for i in range(window, len(sentiment_score) + 1)]
    agg_score = [sentiment_score[i - window : i].mean() for i in range(window, len(sentiment_score) + 1)]
    agg_index = [(index[i-1] - index[i-window])/index[i-window] for i in range(window, len(index) + 1)]

    return pd.DataFrame(zip(agg_date,agg_score, agg_index), columns=['agg_date', 'agg_score', 'agg_index']), stats.pearsonr(agg_score, agg_index)


#코랩용 코드
results = list()
for window in range(1, 6):
  print('#############윈도우 사이즈: ' + str(window) + '################')
  topic = list()
  for i in range(10):
      print(f'#######topic: {i}#######')
      df = pd.read_csv(
          '/content/drive/MyDrive/Colab Notebooks/BERT/SA/final_result/0616_result/' + 'result_topic_' + str(
              i) + '_text.csv', lineterminator='\n')
      result = get_correlation(df, snp, window)
      print()
      print(result)
      print()
      topic.append(result)
  results.append(topic)
#결과는 [window_size-1:[topic-1:(df, corr), topic-2:(df, corr), topic-3:(df, corr),...],
#window_size-2:[topic-1:(df, corr), topic-2:(df, corr), topic-3:(df, corr),...],
#window_size-3:[topic-1:(df, corr), topic-2:(df, corr), topic-3:(df, corr),...],...]
print(results)



#############윈도우 사이즈: 1################
#######topic: 0#######

(      agg_date  agg_score  agg_index
0   2023-05-24  -0.102869  -0.007319
1   2023-05-25  -0.117356   0.008758
2   2023-05-26  -0.097653   0.013049
3   2023-05-27  -0.112554   0.000017
4   2023-05-28  -0.123016   0.000000
5   2023-05-29  -0.101585   0.000000
6   2023-05-30  -0.077469   0.000000
7   2023-05-31  -0.058780  -0.006109
8   2023-06-01  -0.071934   0.009854
9   2023-06-02  -0.081147   0.014534
10  2023-06-03  -0.075893  -0.000701
11  2023-06-04  -0.073019  -0.000701
12  2023-06-05  -0.012021  -0.000603
13  2023-06-06  -0.001172   0.002354
14  2023-06-07  -0.051607  -0.003812
15  2023-06-08   0.005052   0.006189
16  2023-06-09  -0.001519   0.001148
17  2023-06-10  -0.078697   0.000000
18  2023-06-11  -0.077288   0.000000
19  2023-06-12  -0.084362   0.009321
20  2023-06-13  -0.100182   0.006933
21  2023-06-14  -0.087756   0.000819
22  2023-06-15  -0.088365   0.012178, PearsonRResult(statistic=-0.11816233728060814,